### Thư viện sử dụng

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import math

In [2]:
dataset_ratings=pd.read_csv("ratings_Beauty.csv")

Xem số dòng và cột

In [3]:
dataset_ratings.shape

(493800, 4)

Số dòng dữ liệu cho từng cột

In [4]:
dataset_ratings.count()

UserId       493800
ProductId    493800
Rating       493800
Timestamp    493800
dtype: int64

Trình bày tập dữ liệu

In [5]:
dataset_ratings.head

<bound method NDFrame.head of                 UserId   ProductId  Rating   Timestamp
0       A39HTATAQ9V7YF  0205616461     5.0  1369699200
1       A3JM6GV9MNOF9X  0558925278     3.0  1355443200
2       A1Z513UWSAAO0F  0558925278     5.0  1404691200
3       A1WMRR494NWEWV  0733001998     4.0  1382572800
4       A3IAAVS479H7M7  0737104473     1.0  1274227200
...                ...         ...     ...         ...
493795   AEP1F83XRQKNB  B00FGJGNGU     2.0  1391212800
493796  A26A7Q6R1GMCFP  B00FGJGNGU     5.0  1403395200
493797  A3J15VAR3I5RJZ  B00FGJGNGU     5.0  1385683200
493798   ALVJJCDAH5SXB  B00FGJJ9OI     5.0  1397779200
493799  A22UH3YMSE8QFF  B00FGJJ9OI     4.0  1397088000

[493800 rows x 4 columns]>

Trình bày 5 dòng dữ liệu đầu tiên

In [6]:
dataset_ratings.head(5)

,UserId,ProductId,Rating,Timestamp
0,A39HTATAQ9V7YF,0205616461,5.0,1369699200
1,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
2,A1Z513UWSAAO0F,0558925278,5.0,1404691200
3,A1WMRR494NWEWV,0733001998,4.0,1382572800
4,A3IAAVS479H7M7,0737104473,1.0,1274227200


Trình bày 5 dòng dữ liệu cuối cùng

In [7]:
dataset_ratings.tail(5)

,UserId,ProductId,Rating,Timestamp
493795,AEP1F83XRQKNB,B00FGJGNGU,2.0,1391212800
493796,A26A7Q6R1GMCFP,B00FGJGNGU,5.0,1403395200
493797,A3J15VAR3I5RJZ,B00FGJGNGU,5.0,1385683200
493798,ALVJJCDAH5SXB,B00FGJJ9OI,5.0,1397779200
493799,A22UH3YMSE8QFF,B00FGJJ9OI,4.0,1397088000


### Kiểm dữ liệu NULL và NA trong dataset

In [8]:
dataset_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493800 entries, 0 to 493799
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   UserId     493800 non-null  object 
 1   ProductId  493800 non-null  object 
 2   Rating     493800 non-null  float64
 3   Timestamp  493800 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 15.1+ MB


In [9]:
dataset_ratings.isnull().any()

UserId       False
ProductId    False
Rating       False
Timestamp    False
dtype: bool

### Xóa dữ liệu NULL

In [10]:
new_dataset_rating = dataset_ratings.dropna()

Dữ liệu còn lại sau khi xóa NULL

In [11]:
new_dataset_rating.count()

UserId       493800
ProductId    493800
Rating       493800
Timestamp    493800
dtype: int64

In [12]:
new_dataset_rating.shape

(493800, 4)

In [13]:
#Kiểm tra các kiểu dữ liệu
new_dataset_rating.dtypes

UserId        object
ProductId     object
Rating       float64
Timestamp      int64
dtype: object

In [14]:
#Kiểm tra thống kê mô tả
new_dataset_rating.describe().T

,count,mean,std,min,25%,50%,75%,max
Rating,493800.0,4.089113e+00,1.327769e+00,1.000000e+00,4.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00
Timestamp,493800.0,1.380893e+09,1.796930e+07,1.085098e+09,1.368576e+09,1.384042e+09,1.395101e+09,1.406074e+09


In [15]:
#Xem thống kê mô tả cột rating
new_dataset_rating.Rating.describe().T

count    493800.000000
mean          4.089113
std           1.327769
min           1.000000
25%           4.000000
50%           5.000000
75%           5.000000
max           5.000000
Name: Rating, dtype: float64

Khởi tạo SPARK

In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math

import findspark
findspark.init()

import pyspark
findspark.find()

from pyspark.sql import SparkSession
from pyspark.sql.functions import count

from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

spark = (SparkSession
         .builder
         .appName("ALS recommendation spark session")
         .getOrCreate())

Đọc dữ liệu

In [17]:
ratingData = (spark.read
          .option("HEADER", True)
          .option("inferSchema", True)
          .csv("ratings_Beauty.csv")
         )

ratingData.show(5)

+--------------+----------+------+----------+
|        UserId| ProductId|Rating| Timestamp|
+--------------+----------+------+----------+
|A39HTATAQ9V7YF|0205616461|   5.0|1369699200|
|A3JM6GV9MNOF9X|0558925278|   3.0|1355443200|
|A1Z513UWSAAO0F|0558925278|   5.0|1404691200|
|A1WMRR494NWEWV|0733001998|   4.0|1382572800|
|A3IAAVS479H7M7|0737104473|   1.0|1274227200|
+--------------+----------+------+----------+
only showing top 5 rows



Lấy ra những cột cần thiết cho model

In [18]:
nd=ratingData.select(ratingData['UserId'],ratingData['Rating'],ratingData['ProductId'])
nd.show(5)

+--------------+------+----------+
|        UserId|Rating| ProductId|
+--------------+------+----------+
|A39HTATAQ9V7YF|   5.0|0205616461|
|A3JM6GV9MNOF9X|   3.0|0558925278|
|A1Z513UWSAAO0F|   5.0|0558925278|
|A1WMRR494NWEWV|   4.0|0733001998|
|A3IAAVS479H7M7|   1.0|0737104473|
+--------------+------+----------+
only showing top 5 rows



Chuyển cột UserId và ProductId từ (kiểu string) thành label1 (kiểu int)

In [19]:

indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") 
for column in list(set(nd.columns)-set(['Rating'])) ]
pipeline = Pipeline(stages=indexer)
transformed = pipeline.fit(nd).transform(nd)
transformed.show(10)

+--------------+------+----------+---------------+------------+
|        UserId|Rating| ProductId|ProductId_index|UserId_index|
+--------------+------+----------+---------------+------------+
|A39HTATAQ9V7YF|   5.0|0205616461|        43788.0|     49380.0|
|A3JM6GV9MNOF9X|   3.0|0558925278|        28932.0|    263677.0|
|A1Z513UWSAAO0F|   5.0|0558925278|        28932.0|    142369.0|
|A1WMRR494NWEWV|   4.0|0733001998|        43789.0|    137071.0|
|A3IAAVS479H7M7|   1.0|0737104473|        43790.0|    260907.0|
| AKJHHD5VEH7VG|   5.0|0762451459|        43791.0|       186.0|
|A1BG8QW55XHN6U|   5.0|1304139212|        43792.0|       619.0|
|A22VW0P4VZHDE3|   5.0|1304139220|        43793.0|      2034.0|
|A3V3RE4132GKRO|   5.0|130414089X|        43794.0|    288317.0|
|A327B0I7CYTEJC|   4.0|130414643X|        21588.0|    226073.0|
+--------------+------+----------+---------------+------------+
only showing top 10 rows



Khởi tạo sparkDataFrame từ pandas dataFrame

In [20]:
(train, test) = transformed.randomSplit([.8, .2], seed = 1)

 Xây dựng và đánh giá mô hình ALS trên pyspark.

In [21]:
# Xây dựng mô hình recommendation sử dụng thuật toán ALS trên tập dữ liệu huấn luyện

als=ALS(maxIter=5,regParam=0.09,rank=25,userCol="UserId_index",itemCol="ProductId_index",
ratingCol="Rating",coldStartStrategy="drop",nonnegative=True)
model=als.fit(train)

 Xây dựng và đánh giá mô hình ALS trên pyspark.
 Tính RMSE


In [22]:
# Xây dựng mô hình recommendation sử dụng thuật toán ALS trên tập dữ liệu huấn luyện
evaluator=RegressionEvaluator(metricName="rmse",labelCol="Rating",predictionCol="prediction")
predictions=model.transform(test)
rmse=evaluator.evaluate(predictions)
print("RMSE="+str(rmse))
predictions.show(10)

RMSE=2.2446720832484672
+--------------+------+----------+---------------+------------+----------+
|        UserId|Rating| ProductId|ProductId_index|UserId_index|prediction|
+--------------+------+----------+---------------+------------+----------+
| A96FRN3LGYUX9|   5.0|B005XIDZHO|           26.0|       451.0| 2.6961513|
|A25G3M46N5BEW2|   5.0|B005XIDZHO|           26.0|      8782.0| 0.8392905|
|A1H691MQUAA31K|   5.0|B005XIDZHO|           26.0|     13960.0| 1.0437859|
|A38JDRJ4SK0KN2|   5.0|B005XIDZHO|           26.0|     49061.0|0.68080944|
| AICDTM94WTGZH|   5.0|B005XIDZHO|           26.0|     60369.0| 2.8161988|
| ANKUJGLW1TFXH|   5.0|B005XIDZHO|           26.0|     61933.0| 1.0199922|
|A256NW4OC7466Z|   5.0|B005XIDZHO|           26.0|      2768.0| 1.0655527|
|A3A7WDJ5FF4BD1|   1.0|B005XIDZHO|           26.0|      3045.0| 1.1487874|
|A2WHUNQ6R2NBHH|   3.0|B005XIDZHO|           26.0|      9783.0| 3.0934532|
|A349QJY8WW4TB5|   5.0|B005XIDZHO|           26.0|     10070.0| 0.3892031|
+

In [23]:
user_recs=model.recommendForAllUsers(5).show(10)

+------------+--------------------+
|UserId_index|     recommendations|
+------------+--------------------+
|          26|[{35445, 5.877299...|
|          27|[{19594, 9.404769...|
|          28|[{8821, 7.933512}...|
|          31|[{13806, 5.938756...|
|          34|[{11259, 9.141417...|
|          44|[{17345, 9.452127...|
|          53|[{22391, 11.76619...|
|          65|[{12394, 10.64810...|
|          76|[{13038, 10.90148...|
|          78|[{24063, 9.595101...|
+------------+--------------------+
only showing top 10 rows



In [24]:
import pandas as pd
recs=model.recommendForAllUsers(3).toPandas()

In [25]:
nrecs=recs.recommendations.apply(pd.Series) \
            .merge(recs, right_index = True, left_index = True) \
            .drop(["recommendations"], axis = 1) \
            .melt(id_vars = ['UserId_index'], value_name = "recommendation") \
            .drop("variable", axis = 1) \
            .dropna() 
nrecs=nrecs.sort_values('UserId_index')
nrecs=pd.concat([nrecs['recommendation'].apply(pd.Series), nrecs['UserId_index']], axis = 1)
nrecs.columns = [
        
        'ProductId_index',
        'Rating',
        'UserId_index'
       
     ]

In [27]:

md=transformed.select(transformed['UserId'],transformed['UserId_index'],transformed['ProductId'],transformed['ProductId_index'])
md=md.toPandas()
dict1 =dict(zip(md['UserId_index'],md['UserId']))


In [ ]:
dict2=dict(zip(md['ProductId_index'],md['ProductId']))


In [ ]:
nrecs['UserId']=nrecs['UserID_index'].map(dict1)
nrecs['ProductId']=nrecs['ProductID_index'].map(dict2)
 
nrecs=nrecs.sort_values('UserId')
nrecs.reset_index(drop=True, inplace=True)

Kết quả dự đoán

In [ ]:


new=nrecs[['UserId','ProductId','Rating']]
new['recommendations'] = list(zip(new.ProductId, new.Rating))
res=new[['UserId','recommendations']]  
res_new=res['recommendations'].groupby([res.UserId]).apply(list).reset_index()
print(res_new)

KeyboardInterrupt: 